In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [4]:
readfile = '../TestEye.jpg' 

# One Frame Analysis

## Set up
gather information from Frame to analyze
constant for plotting
and basic image manipulation for analusis (gray image and blur)

In [6]:
cap = cv2.VideoCapture(readfile)
read_flag, frame = cap.read()
cap.release()

# frame = cv2.imread(readfileT)

FR=frame.copy()

w, h = frame.shape[:2]

index = -1
thickness = 2
color1 = (0,0,255)
color2 = (255,0,0)
thresh=59
kernel = (7,7)

gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
#CLAHE equalization to increase contrast
cl1 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
clahe = cl1.apply(gray)

blur = cv2.GaussianBlur(clahe,kernel,0)
mblur = cv2.medianBlur(clahe,11)
sblur = cv2.GaussianBlur(clahe,(21,21),0)



if True:
    cv2.imshow('Blur',mblur)
    cv2.imshow('CLAHE',clahe)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Eyelib detection and mask creation

 - find edges
 - define 4 windows for eyelid edge detection
 - find contours over the 4 windows
 - consolidate Top contours and Bottom contours
 - Fit parabols and create area of Interest

In [5]:
edges = cv2.Canny(sblur,120,170,apertureSize = 7)

# region of interest (ROI)
ww=int(w/5)
hh=int(h/2-5)

if True:
    c1 = (int(w/4-ww),int(0))#Top-left
    c2 = (int(w/4-ww),int(h)-hh)#Bottom-left
    c3 = (int(3*w/4-ww),int(0))#Top-right
    c4 = (int(3*w/4-ww),int(h)-hh)#Bottom-right

## creating masks and contours over ROI
def ConMask(canny, c, size):
    w,h=canny.shape
    ww,hh=size
    mask = np.zeros((w, h, 1), dtype='uint8')
    cv2.rectangle(mask,c,(c[0]+ww,c[1]+hh),255,-1)
    Ed = cv2.bitwise_and(canny,canny, mask=mask)
    cc = cv2.findContours(Ed,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnt = cc[0][np.argmax([len(i) for i in cc[0]])]
    
    return cnt


cnt1 = ConMask(edges, c1, (ww,hh))
cnt2 = ConMask(edges, c2, (ww,hh))
cnt3 = ConMask(edges, c3, (2*ww,hh))
cnt4 = ConMask(edges, c4, (2*ww,hh))


## X and Y of all contours
xTop = [k[0][0] for k in cnt1]
xTop.extend([k[0][0] for k in cnt3])
yTop = [k[0][1] for k in cnt1]
yTop.extend([k[0][1] for k in cnt3])

xBot = [k[0][0] for k in cnt2]
xBot.extend([k[0][0] for k in cnt4])
yBot = [k[0][1] for k in cnt2]
yBot.extend([k[0][1] for k in cnt4])


## polyfit
def ParaFit(x,y,width,buffer):
    coef = np.polyfit(x,y,2)
    xTh=np.arange(0,width, dtype='int')
    pTh = np.poly1d(coef)
    yTh = pTh(xTh)
    
    i=int(buffer)
    yTh[yTh<50] = yTh[yTh<50]+i
    yTh[yTh>=50] = yTh[yTh>=50]-i
    
    return np.dstack((xTh,yTh.astype(int)))

buffer=3 # number of pixels added to edges for better mask

Top = ParaFit(xTop,yTop,w,buffer)
Bot = ParaFit(xBot,yBot,w,buffer)
EyeLid = np.vstack((Top.reshape(Top.shape[1],2),
                    np.flip(Bot.reshape(Bot.shape[1],2),0)))

## mask for EyeLid - eMask
eMask = np.zeros((w, h, 1), dtype='uint8')
cv2.fillPoly(eMask,[EyeLid],255)

## mask for Reflexions - rMask
negFR = cv2.bitwise_not(gray)
_, rMask = cv2.threshold(negFR,thresh,255,cv2.THRESH_BINARY)
# erosion for larger mask
kM = np.ones((buffer,buffer),np.uint8)
rMask = cv2.erode(rMask,kM,iterations = 1)

Mask = cv2.bitwise_and(eMask,rMask)
nFR = cv2.bitwise_and(gray,gray, mask=Mask)
nBlur = cv2.bitwise_and(sblur,sblur, mask=cv2.bitwise_not(Mask))

if True:
    ## draw the contours
    cv2.drawContours(FR,[EyeLid], index,color1,1)
    cv2.drawContours(FR, cnt1, index, color2, 2)
    cv2.drawContours(FR, cnt2, index, color2, 2)
    cv2.drawContours(FR, cnt3, index, color2, 2)
    cv2.drawContours(FR, cnt4, index, color2, 2)
    
    cv2.imshow('Eyelid Contours',FR)
    cv2.imshow('Reflexions',rMask)
    cv2.imshow('Mask',Mask)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Pupil detection

In [ ]:
## Pupil detection
_, pupil = cv2.threshold(gray,thresh,255,cv2.THRESH_BINARY_INV)
contoursP, _ = cv2.findContours(pupil, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for c in contoursP:
    if len(c)<4:
        continue
    
    area = cv2.contourArea(c)
    
    if area<100:
        continue
    
    peri = cv2.arcLength(c, True)
    AA = area
    M = cv2.moments(c)# check moments of an image for centroids calaculation
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    
## Polar Transform
radius = np.array([cx, w-cx]).max()
Mag = w/np.log(radius)

grayPL = cv2.logPolar(gray,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS)
rGray_PL = cv2.rotate(grayPL, cv2.ROTATE_90_CLOCKWISE)

maskPl = cv2.logPolar(Mask,(cx,cy),Mag, flags=cv2.INTER_LINEAR + cv2.WARP_FILL_OUTLIERS)
rMask_PL = cv2.rotate(maskPl, cv2.ROTATE_90_CLOCKWISE)


## max Pupil size over planar representation - mPSz
ePL = cv2.Canny(rGray_PL,50,150,apertureSize = 3)
cc = cv2.findContours(ePL,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cntPl = cc[0][np.argmax([len(i) for i in cc[0]])]
mPSz = np.array([k[0][1] for k in cntPl]).max()

ResG = rGray_PL[mPSz+buffer:,:w]
ResM = rMask_PL[mPSz+buffer:,:w]

if False:
    cv2.drawContours(FR, c, index, color1, thickness)
    cv2.drawMarker(FR,(cx,cy),color1)
    
    cv2.imshow("Original", frame)
    cv2.imshow('Process1', FR)
    cv2.imshow('Process2', nFR)


    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Sobel Filtering

In [19]:
nRes = cv2.bitwise_and(ResG,ResG,mask=ResM)

def getGradientMagnitude(im):
    "Get magnitude of gradient for given image"
    ddepth = cv2.CV_32F
    dx = cv2.Sobel(im, ddepth, 1, 0)
    dy = cv2.Sobel(im, ddepth, 0, 1)
    dxabs = cv2.convertScaleAbs(dx)
    dyabs = cv2.convertScaleAbs(dy)
    mag = cv2.addWeighted(dxabs, 0.5, dyabs, 0.5, 0)
    return mag

Mg = getGradientMagnitude(ResG)


if True:
    sobelx = cv2.Sobel(ResG,cv2.CV_32F, 1, 0, 1)
    abs_sobelx = np.absolute(sobelx)
    fx = abs_sobelx.copy()
    fx[fx>55]=0.0
    
    sobelx_8u = np.uint8(abs_sobelx)
    sobel_fx = np.uint8(fx)

    sobely = cv2.Sobel(ResG,cv2.CV_32F, 0, 1, 1)
    abs_sobely = np.absolute(sobely)
    fy = abs_sobely.copy()
    fy[fy>80]=0.0
    fy[fy<30]=0.0

    sobely_8u = np.uint8(abs_sobely)
    sobel_fy = np.uint8(fy)

    ResSob = cv2.magnitude(fx,fy)
    abs_sob = np.absolute(ResSob)
    sob_8u = np.uint8(abs_sob)


if True:
    cv2.drawContours(FR, c, index, color1, thickness)
    cv2.drawMarker(FR,(cx,cy),color1)

#     plt.figure()
#     plt.subplot(3,1,1)
#     plt.imshow(sobelx_8u,cmap='gray')
#     plt.subplot(3,1,2)
#     plt.imshow(sobely_8u,cmap='gray')
#     plt.subplot(3,1,3)
#     plt.imshow(Mg,cmap='gray')

    cv2.imshow('Sobel x',sobel_fx)
    cv2.imshow('Sobel y',sobel_fy)
    cv2.imshow('Sobel',sob_8u)
    
    plt.imshow(ResSob,cmap='gray')

#     cv2.imshow('Sobel',Mg)
#     cv2.imshow('ResG',ResG)
#     cv2.imshow('ResM',ResM)
    cv2.imshow('Result',nRes)
    # cv2.imwrite('Sobel.png',sob_8u)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Full Video Analysis

In [ ]:
index = -1
thickness = 2
color = (255,0,255) #for pink

thresh=59

cap = cv2.VideoCapture(readfile)
Area_raw=[]
Area_approx=[]
Area_ellipse=[]

while True:
    read_flag, frame = cap.read()
    
    if not read_flag:
        break
        
    gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    _, CVbinary = cv2.threshold(gray,thresh,255,cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(CVbinary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        area = cv2.contourArea(c)
        if len(c) > 4 and area>100:
            peri = cv2.arcLength(c, True)

            M = cv2.moments(c)# check moments of an image for centroids calaculation
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])

            approx = cv2.approxPolyDP(c, 0.001 * peri, True)

            #fit ellipse
            ellipse = cv2.fitEllipseDirect(c)
            poly = cv2.ellipse2Poly((int(ellipse[0][0]), int(ellipse[0][1])), 
                                    (int(ellipse[1][0] / 2), int(ellipse[1][1] / 2)), 
                                    int(ellipse[2]), 
                                    0, 360, 5)

            #compute areas
            areaA = cv2.contourArea(approx)
            areaE = cv2.contourArea(poly)


#                 cv2.drawContours(frame, approx, index, color, thickness)
            cv2.ellipse(frame,ellipse,color, thickness)
            Area_raw.append(area)
            Area_approx.append(areaA)
            Area_ellipse.append(areaE)

    if True:
#             cv2.imshow("Frame", frame)
#             cv2.imshow("Gray", gray)
#             cv2.imshow("Thresh", CVbinary)
        cv2.imshow("Contours", frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
Fr = 120
T=np.linspace(1/Fr,len(Area_raw)/Fr,len(Area_raw))
T = T.tolist()
N=20
nData = np.convolve(Area_raw, np.ones((N,))/N, mode='same')
plt.plot(T,nData)
# plt.ylim(500,1500)

In [ ]:
T

In [ ]:
Area_raw

In [ ]:
len(T)